Importing the module 

In [1]:
import pandas as pd

Loading the csv file

In [6]:
commentary_df = pd.read_csv('rcb_commentary.csv')
commentary_df.drop(columns=['match_id','teams','over','timestamp'],inplace=True)
commentary_df.head()

,commentary
0,"Johnson to Livingstone, FOUR, emphatic win for..."
1,"Johnson to Livingstone, SIX, Livingstone is in..."
2,"Arora to Livingstone, 1 run, length delivery o..."
3,"Arora to Livingstone, no run, slower-ball on a..."
4,"Arora to Livingstone, FOUR, Livingstone is off..."


Lowercasing

In [9]:
commentary_df['commentary']=commentary_df['commentary'].str.lower()
print(commentary_df.size)

1518
